In [ ]:
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn import svm
from scipy import signal
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

import matplotlib.pyplot as plt
import scipy.io as sio

!pip install neurokit2
import neurokit2 as nk

import seaborn as sns

import pandas as pd
import numpy as np
import time
!pip install mne
import mne

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
path = "/content/drive/My Drive/DREAMERedit.mat"
raw = sio.loadmat(path)

In [ ]:
#Step 1 : frames


def preprocess_EEG(raw, feature):
 
  frame =[]
  #Dividing into frames
  
  for i in range(0,7680):
    j = i//640
    k = i%640
    frame[j][k] = raw[i]
  
  #frame[0][0-639] - Frame 1, frame[11][0-639] - Frame 12
    fe=[]
    fe.append(deltamean[i])
    fe.append(thetamean[i])
    fe.append(alphamean[i])
    fe.append(betamean[i])
    fe.append(gammamean[i])
      
    feature.append(fe)

  return feature

In [ ]:
participant =0
video = 0
electrode = 0
B, S = [], []
                
basl = (raw["DREAMER"][0, 0]["Data"][0, participant]["EEG"][0, 0]["baseline"][0, 0][video, 0][:, electrode])
stim = (raw["DREAMER"][0, 0]["Data"][0, participant]["EEG"][0, 0]["stimuli"][0, 0][video, 0][:, electrode])
                
#B = preprocess_EEG(basl, B) 
#S = preprocess_EEG(stim, S)
stim.shape

(7680,)

In [ ]:
def feat_extract_EEG(raw):
    
    EEG_tmp = np.zeros((23, 18, 107520))
    for participant in range(0, 23):
        for video in range(0, 18):
          column=0
          for electrode in range(0,14):
            stim = (raw["DREAMER"][0, 0]["Data"][0, participant]["EEG"][0, 0]["stimuli"][0, 0][video, 0][:, electrode])
            for i in range(0,7680):
              EEG_tmp[participant,video,column]= stim[i]
              column+=1
                                

    col = []

    for i in range(0, 7680):
      col.append("Electrode1_"+str(i + 1))
    for i in range(0, 7680):
      col.append("Electrode2_"+str(i + 1))
    for i in range(0, 7680):
      col.append("Electrode3_"+str(i + 1))
    for i in range(0, 7680):
     col.append("Electrode4_"+str(i + 1))
    for i in range(0, 7680):
      col.append("Electrode5_"+str(i + 1))
    for i in range(0, 7680):
      col.append("Electrode6_"+str(i + 1))
    for i in range(0, 7680):
      col.append("Electrode7_"+str(i + 1))
    for i in range(0, 7680):
      col.append("Electrode8_"+str(i + 1))
    for i in range(0, 7680):
     col.append("Electrode9_"+str(i + 1))
    for i in range(0, 7680):
      col.append("Electrode10_"+str(i + 1))
    for i in range(0, 7680):
      col.append("Electrode11_"+str(i + 1))
    for i in range(0, 7680):
      col.append("Electrode12_"+str(i + 1))
    for i in range(0, 7680):
      col.append("Electrode13_"+str(i + 1))
    for i in range(0, 7680):
     col.append("Electrode14_"+str(i + 1))


    EEG_tmp = EEG_tmp.reshape(-1,EEG_tmp.shape[2])
    data_EEG = pd.DataFrame(EEG_tmp, columns=col)
    print(data_EEG)
    from sklearn import preprocessing

    x = data_EEG.values
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    data_EEG = pd.DataFrame(x_scaled)
    data_EEG.columns = col
    print(data_EEG)
    return data_EEG

In [ ]:
def participant_affective(raw):
    a = np.zeros((23, 18, 9), dtype=object)
    for participant in range(0, 23):
        for video in range(0, 18):
            a[participant, video, 0] = (raw["DREAMER"][0, 0]["Data"]
                                        [0, participant]["Age"][0][0][0])
            
            a[participant, video, 1] = (raw["DREAMER"][0, 0]["Data"]
                                        [0, participant]["Gender"][0][0][0])
            
            a[participant, video, 2] = int(participant+1)
            
            a[participant, video, 3] = int(video+1)
            
            a[participant, video, 4] = ["Searching for Bobby Fischer",
                                        "D.O.A.", "The Hangover", "The Ring",
                                        "300", "National Lampoon\'s VanWilder",
                                        "Wall-E", "Crash", "My Girl",
                                        "The Fly", "Pride and Prejudice",
                                        "Modern Times", "Remember the Titans",
                                        "Gentlemans Agreement", "Psycho",
                                        "The Bourne Identitiy",
                                        "The Shawshank Redemption",
                                        "The Departed"][video]
            
            a[participant, video, 5] = ["calmness", "surprise", "amusement",
                                        "fear", "excitement", "disgust",
                                        "happiness", "anger", "sadness",
                                        "disgust", "calmness", "amusement",
                                        "happiness", "anger", "fear",
                                        "excitement", "sadness",
                                        "surprise"][video]
            a[participant, video, 6] = int(raw["DREAMER"][0, 0]["Data"]
                                           [0, participant]["ScoreValence"]
                                           [0, 0][video, 0])
            a[participant, video, 7] = int(raw["DREAMER"][0, 0]["Data"]
                                           [0, participant]["ScoreArousal"]
                                           [0, 0][video, 0])
            a[participant, video, 8] = int(raw["DREAMER"][0, 0]["Data"]
                                           [0, participant]["ScoreDominance"]
                                           [0, 0][video, 0])
    b = pd.DataFrame(a.reshape((23*18, a.shape[2])),
                     columns=["age", "gender", "participant",
                              "video", "video_name", "target_emotion",
                              "valence", "arousal", "dominance"])
    return b

In [ ]:
def_EEG3 = feat_extract_EEG(raw)
#pd.def_EEG3

     Electrode1_1  Electrode1_2  ...  Electrode14_7679  Electrode14_7680
0     4376.923077   4380.512821  ...       4206.666667       4133.333333
1     4396.923077   4394.358974  ...       4180.000000       4191.794872
2     4387.692308   4382.051282  ...       4007.179487       3944.615385
3     4396.923077   4401.538462  ...       4188.205128       4184.615385
4     4398.461538   4395.897436  ...       4140.000000       4140.000000
..            ...           ...  ...               ...               ...
409   4356.410256   4360.512821  ...       4201.538462       4208.205128
410   4381.538462   4377.435897  ...       4193.333333       4214.358974
411   4355.384615   4358.974359  ...       4181.025641       4186.666667
412   4381.538462   4382.051282  ...       4184.102564       4180.512821
413   4397.948718   4395.384615  ...       4142.051282       4153.846154

[414 rows x 107520 columns]
     Electrode1_1  Electrode1_2  ...  Electrode14_7679  Electrode14_7680
0        0.711454     

In [ ]:
#Enter desired electrode
df_features = def_EEG3
#print(df_features)
#print(df_participant_affective)
df_participant_affective = participant_affective(raw)

# not really clear to me why I have to do this because I thought these were already integers
df_participant_affective["valence"] = (df_participant_affective
                                       ["valence"].astype(int))
df_participant_affective["arousal"] = (df_participant_affective
                                       ["arousal"].astype(int))
df_participant_affective["dominance"] = (df_participant_affective
                                         ["dominance"].astype(int))
df = pd.concat([df_features, df_participant_affective], axis=1)
df

,Electrode1_1,Electrode1_2,Electrode1_3,Electrode1_4,Electrode1_5,Electrode1_6,Electrode1_7,Electrode1_8,Electrode1_9,Electrode1_10,Electrode1_11,Electrode1_12,Electrode1_13,Electrode1_14,Electrode1_15,Electrode1_16,Electrode1_17,Electrode1_18,Electrode1_19,Electrode1_20,Electrode1_21,Electrode1_22,Electrode1_23,Electrode1_24,Electrode1_25,Electrode1_26,Electrode1_27,Electrode1_28,Electrode1_29,Electrode1_30,Electrode1_31,Electrode1_32,Electrode1_33,Electrode1_34,Electrode1_35,Electrode1_36,Electrode1_37,Electrode1_38,Electrode1_39,Electrode1_40,...,Electrode14_7650,Electrode14_7651,Electrode14_7652,Electrode14_7653,Electrode14_7654,Electrode14_7655,Electrode14_7656,Electrode14_7657,Electrode14_7658,Electrode14_7659,Electrode14_7660,Electrode14_7661,Electrode14_7662,Electrode14_7663,Electrode14_7664,Electrode14_7665,Electrode14_7666,Electrode14_7667,Electrode14_7668,Electrode14_7669,Electrode14_7670,Electrode14_7671,Electrode14_7672,Electrode14_7673,Electrode14_7674,Electrode14_7675,Electrode14_7676,Electrode14_7677,Electrode14_7678,Electrode14_7679,Electrode14_7680,age,gender,participant,video,video_name,target_emotion,valence,arousal,dominance
0,0.711454,0.816407,0.697264,0.623580,0.540883,0.697167,0.723525,0.664447,0.619359,0.600343,0.795455,0.692970,0.752745,0.748237,0.721244,0.812942,0.750268,0.713971,0.738795,0.846925,0.864148,0.750259,0.589057,0.649112,0.851468,0.727535,0.667166,0.513998,0.662799,0.776796,0.568155,0.500076,0.530692,0.573115,0.552580,0.448053,0.403018,0.367629,0.519017,0.452125,...,0.600570,0.535027,0.482174,0.444218,0.429233,0.417938,0.396732,0.375510,0.377816,0.389163,0.370040,0.328767,0.323363,0.343300,0.331148,0.334926,0.332230,0.325925,0.329387,0.333160,0.327799,0.319943,0.317603,0.326645,0.328819,0.315980,0.302303,0.298552,0.304833,0.296585,0.276508,22,male,1,1,Searching for Bobby Fischer,calmness,4,3,2
1,0.720999,0.824642,0.701456,0.629354,0.548922,0.707850,0.731817,0.670715,0.626261,0.608348,0.805158,0.699980,0.760697,0.756772,0.729119,0.823306,0.760343,0.721423,0.744670,0.850936,0.867150,0.756477,0.596873,0.656294,0.853543,0.726749,0.671016,0.519910,0.669217,0.780624,0.573094,0.505563,0.536576,0.578516,0.559275,0.457948,0.411667,0.375157,0.526581,0.463335,...,0.591586,0.538233,0.513864,0.485009,0.447577,0.438965,0.443438,0.438332,0.420980,0.392611,0.379576,0.363888,0.354684,0.351318,0.320224,0.334789,0.346021,0.342594,0.321363,0.313929,0.320664,0.326143,0.317346,0.304665,0.306543,0.308382,0.306202,0.300586,0.291296,0.290441,0.289939,22,male,1,2,D.O.A.,surprise,3,3,1
2,0.716593,0.817322,0.703663,0.635128,0.549264,0.698792,0.725657,0.675416,0.631039,0.604536,0.796221,0.700781,0.766376,0.756215,0.719793,0.815470,0.762701,0.724979,0.743327,0.846702,0.868651,0.761658,0.599035,0.653700,0.849689,0.731988,0.678717,0.520605,0.660331,0.777091,0.579415,0.509069,0.532065,0.572715,0.560654,0.462680,0.410195,0.368465,0.522799,0.465904,...,0.573181,0.501603,0.443002,0.425658,0.417648,0.405843,0.394956,0.385626,0.387274,0.378654,0.362965,0.343049,0.343962,0.363006,0.334972,0.310222,0.308233,0.291096,0.279795,0.281575,0.284084,0.274219,0.263598,0.266709,0.265663,0.255200,0.245400,0.243748,0.253295,0.250620,0.233153,22,male,1,3,The Hangover,amusement,5,4,4
3,0.720999,0.828911,0.705207,0.629726,0.547212,0.706456,0.733239,0.672870,0.626792,0.607395,0.805414,0.702183,0.761833,0.755288,0.728290,0.823812,0.758414,0.718713,0.743831,0.853832,0.871904,0.756891,0.594379,0.655097,0.858879,0.730417,0.671016,0.519388,0.668230,0.783274,0.576650,0.507087,0.536772,0.580116,0.561245,0.457088,0.410379,0.377457,0.529523,0.462868,...,0.593996,0.523123,0.511884,0.506221,0.471713,0.443245,0.439709,0.452884,0.451591,0.416420,0.383420,0.371029,0.385440,0.388557,0.342209,0.333697,0.345470,0.349370,0.325835,0.301585,0.305487,0.314131,0.308602,0.296613,0.289963,0.296550,0.302059,0.316022,0.311127,0.292331,0.288289,22,male,1,4,The Ring,fear,4,3,2
4,0.721733,0.825557,0.699912,0.627119,0.547725,0.706688,0.728500,0.667581,0.624491,0.606442,0.

In [ ]:
data2 = df.loc[(df['target_emotion'] == 'anger') |
                (df['target_emotion'] == 'fear') |
                (df['target_emotion'] == 'calmness') |
                (df['target_emotion'] == 'surprise') |
                (df['target_emotion'] == 'excitement') |
                (df['target_emotion'] == 'amusement') |
                (df['target_emotion'] == 'happiness') |
                (df['target_emotion'] == 'sadness') |
                (df['target_emotion'] == 'disgust')].copy()

d={'surprise': 0, 'excitement': 0, 'amusement': 0, 'happiness': 0, 'fear': 1, 'anger': 1, 'sadness':2, 'disgust':2 ,'calmness': 3}
data2['class'] = data2.target_emotion.map(d)

e={0: 0, 1: 0, 2: 0, 3: 1, 4: 1, 5: 1 }
data2['valencehigh'] = data2.valence.map(e)

f={0: 0, 1: 0, 2: 0, 3: 1, 4: 1, 5: 1 }
data2['arousalhigh'] = data2.arousal.map(e)

# add another column with title 'stress_bin' and give it value 0/1 depending on what emotion it elicits
#data['stress_bin'] = data['target_emotion'].map({'anger': 1, 'fear': 1, 'calmness': 0})

# save features, demographic, and emotion data as csv
data2.to_csv('normalized.csv')

In [ ]:
print("a")

a


In [ ]:
data2 = pd.read_csv("normalized.csv")  
data_2 = data2.head()
data_2  

,Unnamed: 0,Electrode1_1,Electrode1_2,Electrode1_3,Electrode1_4,Electrode1_5,Electrode1_6,Electrode1_7,Electrode1_8,Electrode1_9,Electrode1_10,Electrode1_11,Electrode1_12,Electrode1_13,Electrode1_14,Electrode1_15,Electrode1_16,Electrode1_17,Electrode1_18,Electrode1_19,Electrode1_20,Electrode1_21,Electrode1_22,Electrode1_23,Electrode1_24,Electrode1_25,Electrode1_26,Electrode1_27,Electrode1_28,Electrode1_29,Electrode1_30,Electrode1_31,Electrode1_32,Electrode1_33,Electrode1_34,Electrode1_35,Electrode1_36,Electrode1_37,Electrode1_38,Electrode1_39,...,Electrode14_7653,Electrode14_7654,Electrode14_7655,Electrode14_7656,Electrode14_7657,Electrode14_7658,Electrode14_7659,Electrode14_7660,Electrode14_7661,Electrode14_7662,Electrode14_7663,Electrode14_7664,Electrode14_7665,Electrode14_7666,Electrode14_7667,Electrode14_7668,Electrode14_7669,Electrode14_7670,Electrode14_7671,Electrode14_7672,Electrode14_7673,Electrode14_7674,Electrode14_7675,Electrode14_7676,Electrode14_7677,Electrode14_7678,Electrode14_7679,Electrode14_7680,age,gender,participant,video,video_name,target_emotion,valence,arousal,dominance,class,valencehigh,arousalhigh
0,0,0.711454,0.816407,0.697264,0.623580,0.540883,0.697167,0.723525,0.664447,0.619359,0.600343,0.795455,0.692970,0.752745,0.748237,0.721244,0.812942,0.750268,0.713971,0.738795,0.846925,0.864148,0.750259,0.589057,0.649112,0.851468,0.727535,0.667166,0.513998,0.662799,0.776796,0.568155,0.500076,0.530692,0.573115,0.552580,0.448053,0.403018,0.367629,0.519017,...,0.444218,0.429233,0.417938,0.396732,0.375510,0.377816,0.389163,0.370040,0.328767,0.323363,0.343300,0.331148,0.334926,0.332230,0.325925,0.329387,0.333160,0.327799,0.319943,0.317603,0.326645,0.328819,0.315980,0.302303,0.298552,0.304833,0.296585,0.276508,22,male,1,1,Searching for Bobby Fischer,calmness,4,3,2,3,1,1
1,1,0.720999,0.824642,0.701456,0.629354,0.548922,0.707850,0.731817,0.670715,0.626261,0.608348,0.805158,0.699980,0.760697,0.756772,0.729119,0.823306,0.760343,0.721423,0.744670,0.850936,0.867150,0.756477,0.596873,0.656294,0.853543,0.726749,0.671016,0.519910,0.669217,0.780624,0.573094,0.505563,0.536576,0.578516,0.559275,0.457948,0.411667,0.375157,0.526581,...,0.485009,0.447577,0.438965,0.443438,0.438332,0.420980,0.392611,0.379576,0.363888,0.354684,0.351318,0.320224,0.334789,0.346021,0.342594,0.321363,0.313929,0.320664,0.326143,0.317346,0.304665,0.306543,0.308382,0.306202,0.300586,0.291296,0.290441,0.289939,22,male,1,2,D.O.A.,surprise,3,3,1,0,1,1
2,2,0.716593,0.817322,0.703663,0.635128,0.549264,0.698792,0.725657,0.675416,0.631039,0.604536,0.796221,0.700781,0.766376,0.756215,0.719793,0.815470,0.762701,0.724979,0.743327,0.846702,0.868651,0.761658,0.599035,0.653700,0.849689,0.731988,0.678717,0.520605,0.660331,0.777091,0.579415,0.509069,0.532065,0.572715,0.560654,0.462680,0.410195,0.368465,0.522799,...,0.425658,0.417648,0.405843,0.394956,0.385626,0.387274,0.378654,0.362965,0.343049,0.343962,0.363006,0.334972,0.310222,0.308233,0.291096,0.279795,0.281575,0.284084,0.274219,0.263598,0.266709,0.265663,0.255200,0.245400,0.243748,0.253295,0.250620,0.233153,22,male,1,3,The Hangover,amusement,5,4,4,0,1,1
3,3,0.720999,0.828911,0.705207,0.629726,0.547212,0.706456,0.733239,0.672870,0.626792,0.607395,0.805414,0.702183,0.761833,0.755288,0.728290,0.823812,0.758414,0.718713,0.743831,0.853832,0.871904,0.756891,0.594379,0.655097,0.858879,0.730417,0.671016,0.519388,0.668230,0.783274,0.576650,0.507087,0.536772,0.580116,0.561245,0.457088,0.410379,0.377457,0.529523,...,0.506221,0.471713,0.443245,0.439709,0.452884,0.451591,0.416420,0.383420,0.371029,0.385440,0.388557,0.342209,0.333697,0.345470,0.349370,0.325835,0.301585,0.305487,0.314131,0.308602,0.296613,0.289963,0.296550,0.302059,0.316022,0.311127,0.292331,0.288289,22,male,1,4,The Ring,fear,4,3,2,1,1,1
4,4,0.721733,0.825557,0.699912,0.627119,0.547725,0.706688,0.728500,0.667581,0.624491,0.606442,0.802605,0.698177,0.758425,0.753618,0.727254,0.822548,0.757985,0.718713,0.743999,0.852496,0.866900,0.755440,0.596707,0.655695,0.852060,0

In [ ]:
## SOME CHECKS
# print( type(data))
#data = pd.read_csv("ml_data.csv")  
#data_ = data.head()  


In [ ]:
#DATA PREP FOR VALENCE

#data  - ml_data.csv
#data2 - ml_data_untouched.csv
#Valence is judged by alphas of F3 and F4
#F3 is data 3, F4 is data 12

#df1=data2.reset_index()['psdalpha_12']
#df2=data2.reset_index()['psdalpha_3']
#df1 = np.array(df1, dtype = float)
#df2 = np.array(df2, dtype = float)
#df3 = np.concatenate((df1, df2), axis=0)
#data2=data2.reset_index()
dfz=data2.iloc[:,1:43]
#dfy=pd.DataFrame(data={ 'alpha12':df1,'alpha3':df2})
#df1 = df1.append(df2)
dfz = np.array(dfz, dtype = float)
dfz

In [ ]:
target=data2['valencehigh']
#target = target.append(target)
target

In [ ]:
from sklearn.model_selection import train_test_split
#df1 = np.array(df1, dtype = float)
#df2 = np.array(df2, dtype = float)
target = np.array(target, dtype = float)


x_Train, x_Test, y_Train, y_Test = train_test_split(dfz, target, test_size = 0.35, random_state = 6)

In [ ]:
x_Train = x_Train.reshape(x_Train.shape[0],1,42)
x_Test = x_Test.reshape(x_Test.shape[0],1,42)
x_Test.shape

In [ ]:
import tensorflow.keras.layers as KL
inputs = KL.Input(shape=(1,42))
x = KL.LSTM(units = 8, activation = 'relu')(inputs)
outputs = KL.Dense(units=1, activation= 'sigmoid')(x)

model = tf.keras.models.Model(inputs, outputs)

opt = tf.keras.optimizers.Adamax()
model.compile( optimizer=opt , loss='binary_crossentropy', metrics = ['accuracy'])
history = model.fit(x_Train, y_Train, batch_size=20, epochs=1000, class_weight={0:1, 1:2},validation_data =(x_Test,y_Test))

#y_pred = model.predict(x_Test)
#print(y_pred)


In [ ]:
#from sklearn.metrics import confusion_matrix, classification_report,mean_squared_error
#confudf = pd.DataFrame(confusion_matrix(y_Test,y_pred), index = ["Negative (Actual)","Positive (Actual)"], columns=['Negative (Predicted)', 'Postive (Predicted)'])
#text = np.asarray([['True Negative', 'False Positive'], ['False Negative', 'True Positive']])
#labels = (np.asarray(["{0}\n{1:.2f}".format(text,data) for text, data in zip(text.flatten(), confusion_matrix(y_Test,y_pred).flatten())])).reshape(2,2)


#sns.heatmap(confudf, annot = labels, center=100, fmt='', linewidth = '1', vmax=300, cmap = "Blues", cbar=False)

#plt.yticks(rotation = 0)

In [ ]:
#model for Valence -LSTM

#x_Train = x_Train.reshape(x_Train.shape[0],1,1)
#x_Test = x_Test.reshape(x_Test.shape[0],1,1)

model = Sequential()
model.add(LSTM(128,activation = 'relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64,activation ='relu'))
model.add(Dropout(0.2))

model.add(Dense(32,activation ='relu'))
model.add(Dense(2,activation ='sigmoid'))
y_pred = model.predict(x_Test)
opt = tf.keras.optimizers.Adam(learning_rate=1e-3, decay=1e-5)
model.compile(loss='sparse_categorical_crossentropy',optimizer =opt,metrics=['accuracy'])
#model.compile(loss='binary_crossentropy',optimizer =opt,metrics=['accuracy'])
history = model.fit(x_Train,y_Train,batch_size=16,epochs=200, validation_data =(x_Test,y_Test))
#print(y_pred)

In [ ]:
print(y_pred)

In [ ]:
model.summary()

In [ ]:
#Alternate model for Valence - CNN

#x_Train = x_Train.reshape(x_Train.shape[0],1,1)
#x_Test = x_Test.reshape(x_Test.shape[0],1,1)

model = Sequential()
#model.add(tf.keras.layers.Flatten())
model.add(Dense(128,activation =tf.nn.relu))
model.add(Dropout(0.2))
model.add(Dense(128,activation =tf.nn.relu))
model.add(Dense(10,activation =tf.nn.softmax))


opt = tf.keras.optimizers.Adam(learning_rate=1e-3, decay=1e-5)

model.compile(loss='sparse_categorical_crossentropy',optimizer =opt ,metrics=['accuracy'])
history_CNN = model.fit(x_Train,y_Train,epochs=100, validation_data =(x_Test,y_Test))

In [ ]:
model.summary()

In [ ]:
 plt.plot(history.history['loss'])
 plt.show()
 plt.plot(history_CNN.history['loss'])
 plt.show()


In [ ]:
#DATA PREP FOR AROUSAL

#Arousal is judged by alpha beta ratio of AF3 and AF4
#AF3 is data 1, AF4 is data 14
df3=data2.reset_index()['psdalpha_1']
df4=data2.reset_index()['psdbeta_1']

df34 = df4/df3

df5=data2.reset_index()['psdalpha_14']
df6=data2.reset_index()['psdbeta_14']

df56  = df5/df6

df7 = df34.append(df56)
df7

In [ ]:
target2 =data2['arousalhigh']
#target2 = target2.append(target2)
target2

In [ ]:
from sklearn.model_selection import train_test_split
#df7 = np.array(df7, dtype = float)
target2 = np.array(target2, dtype = float)
x_Train2, x_Test2, y_Train2, y_Test2 = train_test_split(dfz, target2, test_size = 0.35, random_state = 5)

In [ ]:
x_Train2 = x_Train2.reshape(x_Train2.shape[0],1,42)
x_Test2 = x_Test2.reshape(x_Test2.shape[0],1,42)
x_Test2.shape

In [ ]:
#model for Arousal

#x_Train2 = x_Train2.reshape(x_Train2.shape[0],1,1)
#x_Test2 = x_Test2.reshape(x_Test2.shape[0],1,1)

model = Sequential()
model.add(LSTM(128,activation = 'relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128,activation ='relu'))
model.add(Dense(32,activation ='relu'))
model.add(Dense(10,activation ='softmax'))

opt = tf.keras.optimizers.Adam(learning_rate=1e-3, decay=1e-5)
model.compile(loss='sparse_categorical_crossentropy',optimizer =opt,metrics=['accuracy'])
history = model.fit(x_Train2,y_Train2,epochs=1000, validation_data =(x_Test2,y_Test2))

In [ ]:
 plt.plot(history.history['loss'])
 plt.show()